In [1]:
import pandas as pd

In [11]:
data = pd.read_csv('gene_high_throughput_sequencing.csv')

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to 71
Columns: 15750 entries, Patient_id to EIF1AY
dtypes: float64(15748), object(2)
memory usage: 8.7+ MB


In [14]:
data.iloc[:5, :5]

,Patient_id,Diagnosis,LOC643837,LOC100130417,SAMD11
0,STT5425_Breast_001_normal,normal,1.257614,2.408148,13.368622
1,STT5427_Breast_023_normal,normal,4.567931,16.602734,42.477752
2,STT5430_Breast_002_normal,normal,2.077597,3.978294,12.863214
3,STT5439_Breast_003_normal,normal,2.066576,8.520713,14.466035
4,STT5441_Breast_004_normal,normal,2.613616,3.434965,12.682222


In [15]:
data.Diagnosis.value_counts()

early neoplasia    25
normal             24
cancer             23
Name: Diagnosis, dtype: int64

In [17]:
data_normal = data[data.Diagnosis=='normal']
data_cancer = data[data.Diagnosis=='cancer']
data_en = data[data.Diagnosis=='early neoplasia']

In [18]:
from scipy.stats import ttest_ind

In [25]:
ttest_ind(data_normal.loc[:,'LOC643837'], data_en.loc[:,'LOC643837']).pvalue

0.69059747056723886

In [50]:
genes = data.columns[2:]
ps_norm_en = []
ps_en_cancer = []
for gene in genes:
    res = ttest_ind(data_normal.loc[:,gene], data_en.loc[:,gene], equal_var=False)
    ps_norm_en.append(res.pvalue)
    res = ttest_ind(data_en.loc[:,gene], data_cancer.loc[:,gene], equal_var=False)
    ps_en_cancer.append(res.pvalue)

In [51]:
import numpy as np
ps_norm_en = np.array(ps_norm_en)
ps_en_cancer = np.array(ps_en_cancer)

In [52]:
ps_norm_en.shape
genes.shape

(15748L,)

In [53]:
alpha = 0.05

#count = np.array(map(int, ps_norm_en<alpha) + map(int, ps_en_cancer<alpha))

print sum(ps_norm_en<alpha)
print sum(ps_en_cancer<alpha)

1575
3490


In [54]:
with open('task1.1.txt', 'w') as f:
    f.write(str(sum(ps_norm_en<alpha)))
    
with open('task1.2.txt', 'w') as f:
    f.write(str(sum(ps_en_cancer<alpha)))

In [55]:
import statsmodels.stats.multitest as smm

In [56]:
alpha2 = 0.05/2

In [64]:
ps_norm_en_hs = smm.multipletests(ps_norm_en, alpha2)[1]
ps_en_cancer_hs = smm.multipletests(ps_en_cancer, alpha2)[1]

In [65]:
fc_norm_en = np.zeros(ps_norm_en_hs.shape)
fc_en_cancer = np.zeros(ps_en_cancer_hs.shape)
for i in range(ps_norm_en_hs.shape[0]):
    mean_n = data_normal.iloc[:,2+i].mean()
    mean_en = data_en.iloc[:,2+i].mean()
    mean_c = data_cancer.iloc[:,2+i].mean()
    if mean_n>mean_en:
        fc_norm_en[i] = -float(mean_n)/mean_en
    else:
        fc_norm_en[i] = float(mean_en)/mean_n
    
    if mean_en>mean_c:
        fc_en_cancer[i] = -float(mean_en)/mean_c
    else:
        fc_en_cancer[i] = float(mean_c)/mean_en

In [69]:
c1 = ps_norm_en[np.logical_and(ps_norm_en_hs<alpha2, np.abs(fc_norm_en)>1.5)].shape[0]
print c1

2


In [70]:
c2 = ps_en_cancer_hs[np.logical_and(ps_en_cancer_hs<alpha2, np.abs(fc_en_cancer)>1.5)].shape[0]
print c2

77


In [71]:
with open('task2.1.txt', 'w') as f:
    f.write(str(c1))
    
with open('task2.2.txt', 'w') as f:
    f.write(str(c2))

In [72]:
ps_norm_en_bh = smm.multipletests(ps_norm_en, alpha2, method='fdr_bh')[1]
ps_en_cancer_bh = smm.multipletests(ps_en_cancer, alpha2,  method='fdr_bh')[1]

In [73]:
c3 = ps_norm_en_bh[np.logical_and(ps_norm_en_bh<alpha2, np.abs(fc_norm_en)>1.5)].shape[0]
print c3

4


In [74]:
c4 = ps_en_cancer_bh[np.logical_and(ps_en_cancer_bh<alpha2, np.abs(fc_en_cancer)>1.5)].shape[0]
print c4

524


In [75]:
with open('task3.1.txt', 'w') as f:
    f.write(str(c3))
    
with open('task3.2.txt', 'w') as f:
    f.write(str(c4))